# Experiment 003: Python Local Search with Fractional Translation

Implement fine-grained local search to improve the pre-computed solution.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely import affinity
from decimal import Decimal, getcontext
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        # Tree dimensions
        trunk_w, trunk_h = 0.15, 0.2
        base_w, mid_w, top_w = 0.7, 0.4, 0.25
        tip_y, tier_1_y, tier_2_y = 0.8, 0.5, 0.25
        base_y, trunk_bottom_y = 0.0, -trunk_h
        
        # 15 vertices
        initial_polygon = Polygon([
            (0, tip_y),
            (top_w/2, tier_1_y),
            (top_w/4, tier_1_y),
            (mid_w/2, tier_2_y),
            (mid_w/4, tier_2_y),
            (base_w/2, base_y),
            (trunk_w/2, base_y),
            (trunk_w/2, trunk_bottom_y),
            (-trunk_w/2, trunk_bottom_y),
            (-trunk_w/2, base_y),
            (-base_w/2, base_y),
            (-mid_w/4, tier_2_y),
            (-mid_w/2, tier_2_y),
            (-top_w/4, tier_1_y),
            (-top_w/2, tier_1_y),
        ])
        
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def get_bounding_box(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T for t in trees])
    min_xy = xys.min(axis=0)
    max_xy = xys.max(axis=0)
    return min_xy, max_xy

def get_side_length(trees):
    min_xy, max_xy = get_bounding_box(trees)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def get_score(trees, n):
    side = get_side_length(trees)
    return side**2 / n

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def parse_submission(df):
    groups = {}
    for n in range(1, 201):
        group = df[df["id"].str.startswith(f"{n:03d}_")]
        if len(group) > 0:
            trees = []
            for _, row in group.iterrows():
                x = str(row["x"])[1:]
                y = str(row["y"])[1:]
                deg = str(row["deg"])[1:]
                trees.append(ChristmasTree(x, y, deg))
            groups[n] = trees
    return groups

def calculate_total_score(groups):
    total = 0.0
    for n, trees in groups.items():
        total += get_score(trees, n)
    return total

print("Parsing functions defined")

Parsing functions defined


In [4]:
# Load baseline
print("Loading baseline...")
baseline_df = pd.read_csv('/home/code/santa_data/santa-2025.csv')
groups = parse_submission(baseline_df)
initial_score = calculate_total_score(groups)
print(f"Initial score: {initial_score:.6f}")

Loading baseline...


Initial score: 70.734327


In [5]:
# Fractional translation local search
def local_search_config(trees, n, max_iters=100):
    """Apply local search to a single configuration"""
    best_side = get_side_length(trees)
    best_trees = trees.copy()
    
    # Fine-grained steps for fractional translation
    steps = [0.01, 0.005, 0.002, 0.001, 0.0005, 0.0002, 0.0001]
    # 8 directions
    directions = [(1,0), (-1,0), (0,1), (0,-1), (1,1), (1,-1), (-1,1), (-1,-1)]
    # Rotation adjustments
    rotations = [5.0, 2.0, 1.0, 0.5, 0.2, 0.1]
    
    improved = True
    iteration = 0
    
    while improved and iteration < max_iters:
        improved = False
        iteration += 1
        
        for i in range(len(trees)):
            orig_x, orig_y, orig_a = trees[i].center_x, trees[i].center_y, trees[i].angle
            
            # Try position moves
            for step in steps:
                for dx, dy in directions:
                    # Create new tree with moved position
                    new_x = orig_x + dx * step
                    new_y = orig_y + dy * step
                    
                    # Create test configuration
                    test_trees = trees.copy()
                    test_trees[i] = ChristmasTree(str(new_x), str(new_y), str(orig_a))
                    
                    if not has_overlap(test_trees):
                        new_side = get_side_length(test_trees)
                        if new_side < best_side - 1e-10:
                            best_side = new_side
                            best_trees = test_trees
                            trees = test_trees
                            improved = True
                            break
                if improved:
                    break
            
            # Try rotation moves
            if not improved:
                for rot in rotations:
                    for da in [rot, -rot]:
                        new_a = orig_a + da
                        test_trees = trees.copy()
                        test_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(new_a))
                        
                        if not has_overlap(test_trees):
                            new_side = get_side_length(test_trees)
                            if new_side < best_side - 1e-10:
                                best_side = new_side
                                best_trees = test_trees
                                trees = test_trees
                                improved = True
                                break
                    if improved:
                        break
    
    return best_trees, best_side

print("Local search function defined")

Local search function defined


In [6]:
# Test on small n values first (they contribute most to score)
import time

print("Testing local search on small n values...")
test_ns = [2, 3, 4, 5, 6, 7, 8, 9, 10]

for n in test_ns:
    trees = groups[n]
    orig_score = get_score(trees, n)
    
    start = time.time()
    new_trees, new_side = local_search_config(trees, n, max_iters=10)
    elapsed = time.time() - start
    
    new_score = get_score(new_trees, n)
    improvement = orig_score - new_score
    
    if improvement > 1e-10:
        print(f"n={n}: {orig_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
        groups[n] = new_trees
    else:
        print(f"n={n}: {orig_score:.6f} (no improvement) [{elapsed:.1f}s]")

Testing local search on small n values...
n=2: 0.450779 (no improvement) [0.0s]
n=3: 0.434745 (no improvement) [0.0s]
n=4: 0.416545 (no improvement) [0.0s]
n=5: 0.416850 (no improvement) [0.1s]


n=6: 0.399610 (no improvement) [0.1s]
n=7: 0.399897 (no improvement) [0.1s]


n=8: 0.385407 (no improvement) [0.1s]
n=9: 0.387415 (no improvement) [0.2s]


n=10: 0.376630 (no improvement) [0.1s]


In [7]:
# Calculate new total score
new_total = calculate_total_score(groups)
print(f"\nOriginal score: {initial_score:.6f}")
print(f"New score:      {new_total:.6f}")
print(f"Improvement:    {initial_score - new_total:.6f}")


Original score: 70.734327
New score:      70.734327
Improvement:    0.000000


In [8]:
# Try squeeze operation on all configurations
def squeeze_config(trees, n):
    """Try to squeeze configuration toward center"""
    min_xy, max_xy = get_bounding_box(trees)
    cx = (min_xy[0] + max_xy[0]) / 2
    cy = (min_xy[1] + max_xy[1]) / 2
    
    best_side = get_side_length(trees)
    best_trees = trees
    
    for scale in np.arange(0.9999, 0.98, -0.0001):
        test_trees = []
        for t in trees:
            new_x = cx + (t.center_x - cx) * scale
            new_y = cy + (t.center_y - cy) * scale
            test_trees.append(ChristmasTree(str(new_x), str(new_y), str(t.angle)))
        
        if not has_overlap(test_trees):
            new_side = get_side_length(test_trees)
            if new_side < best_side - 1e-10:
                best_side = new_side
                best_trees = test_trees
        else:
            break
    
    return best_trees, best_side

print("Testing squeeze on small n values...")
for n in test_ns:
    trees = groups[n]
    orig_score = get_score(trees, n)
    
    new_trees, new_side = squeeze_config(trees, n)
    new_score = get_score(new_trees, n)
    improvement = orig_score - new_score
    
    if improvement > 1e-10:
        print(f"n={n}: squeezed, improved by {improvement:.6f}")
        groups[n] = new_trees
    else:
        print(f"n={n}: no squeeze improvement")

Testing squeeze on small n values...
n=2: no squeeze improvement
n=3: no squeeze improvement
n=4: no squeeze improvement
n=5: no squeeze improvement
n=6: no squeeze improvement
n=7: no squeeze improvement
n=8: no squeeze improvement
n=9: no squeeze improvement
n=10: no squeeze improvement


In [9]:
# Try other pre-computed solutions
print("Evaluating telegram solutions...")

# Load 71.97 solution
telegram1_df = pd.read_csv('/home/code/telegram_data/71.97.csv')
telegram1_groups = parse_submission(telegram1_df)
telegram1_score = calculate_total_score(telegram1_groups)
print(f"Telegram 71.97.csv score: {telegram1_score:.6f}")

# Load 72.49 solution
telegram2_df = pd.read_csv('/home/code/telegram_data/72.49.csv')
telegram2_groups = parse_submission(telegram2_df)
telegram2_score = calculate_total_score(telegram2_groups)
print(f"Telegram 72.49.csv score: {telegram2_score:.6f}")

In [ ]:
# Final score
final_score = calculate_total_score(groups)
print(f"\nFinal score: {final_score:.6f}")
print(f"Total improvement: {initial_score - final_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {final_score - 68.931058:.6f}")